We will build a character-wise RNN trained on Anna Karenina dataset. It'll be able to generate new text based on the text from the book.

<img src="http://karpathy.github.io/assets/rnn/charseq.jpeg" width="500">

# Import Libraries

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [2]:
# This tutorial uses tensorflow 1.x
tf.__version__

'1.15.0'

# Read Dataset

In [3]:
with open('anna.txt', 'r') as f:
    text=f.read()
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
len(text)

1985223

# Indexing

We'll convert text into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [5]:
# create vocabulary of unique characters
vocab = set(text)
print(vocab)

{'h', 'x', '-', '(', 'a', 'b', '\n', 'P', '@', 'j', '5', 'i', 'z', '0', 'y', '4', '2', '.', 'w', 'v', 'C', 's', 'N', 'r', 'T', 'Y', ')', 'o', ' ', 'V', 'q', 'p', '6', '8', '"', 'K', 'O', '`', 'g', 'Z', ';', 't', 'S', 'M', 'U', '!', '$', 'k', 'H', '&', 'I', ':', 'l', ',', '1', 'm', 'e', 'L', 'n', '/', 'X', 'J', 'D', '*', '7', 'f', '9', '%', 'G', "'", 'Q', 'c', 'W', 'A', 'E', 'B', 'u', 'd', '3', 'F', 'R', '?', '_'}


In [6]:
# indexing of characters in vocabulary
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
print(int_to_vocab)

{0: 'h', 1: 'x', 2: '-', 3: '(', 4: 'a', 5: 'b', 6: '\n', 7: 'P', 8: '@', 9: 'j', 10: '5', 11: 'i', 12: 'z', 13: '0', 14: 'y', 15: '4', 16: '2', 17: '.', 18: 'w', 19: 'v', 20: 'C', 21: 's', 22: 'N', 23: 'r', 24: 'T', 25: 'Y', 26: ')', 27: 'o', 28: ' ', 29: 'V', 30: 'q', 31: 'p', 32: '6', 33: '8', 34: '"', 35: 'K', 36: 'O', 37: '`', 38: 'g', 39: 'Z', 40: ';', 41: 't', 42: 'S', 43: 'M', 44: 'U', 45: '!', 46: '$', 47: 'k', 48: 'H', 49: '&', 50: 'I', 51: ':', 52: 'l', 53: ',', 54: '1', 55: 'm', 56: 'e', 57: 'L', 58: 'n', 59: '/', 60: 'X', 61: 'J', 62: 'D', 63: '*', 64: '7', 65: 'f', 66: '9', 67: '%', 68: 'G', 69: "'", 70: 'Q', 71: 'c', 72: 'W', 73: 'A', 74: 'E', 75: 'B', 76: 'u', 77: 'd', 78: '3', 79: 'F', 80: 'R', 81: '?', 82: '_'}


In [7]:
# Convert text data to numeric format 
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print(chars[:100])

[20  0  4 31 41 56 23 28 54  6  6  6 48  4 31 31 14 28 65  4 55 11 52 11
 56 21 28  4 23 56 28  4 52 52 28  4 52 11 47 56 40 28 56 19 56 23 14 28
 76 58  0  4 31 31 14 28 65  4 55 11 52 14 28 11 21 28 76 58  0  4 31 31
 14 28 11 58 28 11 41 21 28 27 18 58  6 18  4 14 17  6  6 74 19 56 23 14
 41  0 11 58]


Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [8]:
len(vocab)

83

# Make Training and Validation batches

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the batch size. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [9]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # This is a Supervised learning problem
    # For every character prediction is the next character
    # If the word is HELLO, X = H and Y = E, X = E and Y = L and so on
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [10]:
# Example
#-----------
# Total characters = 1786500 (after dropping some characters from the text to make full batches)
# 1786500 characters are divided into batches of 10 (batch_size). Therefore, each batch has 178650 characters.
# from each batch of 178650 characters, send sub-batches of 50 (num_steps) into NN.
train_x, train_y, val_x, val_y = split_data(chars, 10, 50)
train_x.shape

(10, 178650)

I'll write another function to grab batches out of the arrays made by `split_data`. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [11]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

# Model Development

Below is a function where I build the graph for the network.

In [12]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
    '''
    Arguments
    ---------
    num_classes = vocab size
    batch_size - Number of characters/sequences running through the network in one pass.
    num_steps - Number of characters in the sequence the network is trained on. 
                Larger is better typically, the network will learn more long range dependencies. 
                But it takes longer to train. 100 is typically a good number here.
    lstm_size - The number of cells/units in each LSTM layer.
    num_layers - Number of LSTM layers.
    learning_rate - Learning rate for training
    '''
    
    # When we're using this network for sampling later, we'll be passing in
    # one character at a time, so providing an option for that
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # One-hot encoding the input and target characters
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)

    ###################################### Build the RNN layers #############################################
    
    # Basic LSTM layer
    def make_layer(lstm_size):
        return tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple = True)
    
    
    # Stacked RNN
    cell = tf.contrib.rnn.MultiRNNCell([make_layer(lstm_size) for _ in range(num_layers)], state_is_tuple = True)
    
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    

    ### Run the data through the RNN layers
    # This makes a list where each element is on step in the sequence
    rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
    
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one output row for each step for each batch
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # Now connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and batch
    logits = tf.matmul(output, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    preds = tf.nn.softmax(logits, name='predictions')
    
    # Reshape the targets to match the logits
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)

    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    # NOTE: I'm using a namedtuple here because I think they are cool
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Tips and Tricks for Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to write it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).


>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - **Number of parameters in model** - 1 million parameters is 1 million characters. This is printed when you start training.
> - **Size of dataset** - 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - If dataset = 100MB (100 million characters), parameters = 150K (0.15 million characters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - If dataset = 10MB (10 million characters), parameters = 10 million (10 million characters). In this case I'll have to monitor my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


# Training

Time for training which is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}_v{validation loss}.ckpt`

In [13]:
# create a checkpointing directory
!mkdir checkpoints

A subdirectory or file checkpoints already exists.


In [14]:
batch_size = 100
num_steps = 100 
lstm_size = 512
num_layers = 2
learning_rate = 0.001
keep_prob = 10.5
epochs = 2
save_every_n = 200 # Save model every N iterations

In [15]:
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        print('############################# Epoch {} #############################'.format(e+1))
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/i{}_l{}_v{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

############################# Epoch 1 #############################
Epoch 1/2  Iteration 1/356 Training loss: 4.4202 12.7677 sec/batch
Epoch 1/2  Iteration 2/356 Training loss: 4.3765 2.1770 sec/batch
Epoch 1/2  Iteration 3/356 Training loss: 4.2056 2.3061 sec/batch
Epoch 1/2  Iteration 4/356 Training loss: 4.3482 2.0357 sec/batch
Epoch 1/2  Iteration 5/356 Training loss: 4.2881 2.2615 sec/batch
Epoch 1/2  Iteration 6/356 Training loss: 4.1822 2.3661 sec/batch
Epoch 1/2  Iteration 7/356 Training loss: 4.0811 2.2308 sec/batch
Epoch 1/2  Iteration 8/356 Training loss: 3.9894 2.4757 sec/batch
Epoch 1/2  Iteration 9/356 Training loss: 3.9080 2.1525 sec/batch
Epoch 1/2  Iteration 10/356 Training loss: 3.8414 2.3237 sec/batch
Epoch 1/2  Iteration 11/356 Training loss: 3.7829 2.3937 sec/batch
Epoch 1/2  Iteration 12/356 Training loss: 3.7336 2.1655 sec/batch
Epoch 1/2  Iteration 13/356 Training loss: 3.6898 2.4509 sec/batch
Epoch 1/2  Iteration 14/356 Training loss: 3.6530 2.2802 sec/batch
Ep

Epoch 1/2  Iteration 123/356 Training loss: 3.1304 2.3307 sec/batch
Epoch 1/2  Iteration 124/356 Training loss: 3.1281 2.2349 sec/batch
Epoch 1/2  Iteration 125/356 Training loss: 3.1256 2.4167 sec/batch
Epoch 1/2  Iteration 126/356 Training loss: 3.1229 2.3568 sec/batch
Epoch 1/2  Iteration 127/356 Training loss: 3.1204 2.4087 sec/batch
Epoch 1/2  Iteration 128/356 Training loss: 3.1190 2.3709 sec/batch
Epoch 1/2  Iteration 129/356 Training loss: 3.1180 2.2904 sec/batch
Epoch 1/2  Iteration 130/356 Training loss: 3.1160 2.4930 sec/batch
Epoch 1/2  Iteration 131/356 Training loss: 3.1143 2.2336 sec/batch
Epoch 1/2  Iteration 132/356 Training loss: 3.1124 2.3918 sec/batch
Epoch 1/2  Iteration 133/356 Training loss: 3.1103 2.3836 sec/batch
Epoch 1/2  Iteration 134/356 Training loss: 3.1081 2.3647 sec/batch
Epoch 1/2  Iteration 135/356 Training loss: 3.1054 2.5038 sec/batch
Epoch 1/2  Iteration 136/356 Training loss: 3.1030 2.2718 sec/batch
Epoch 1/2  Iteration 137/356 Training loss: 3.10

Epoch 2/2  Iteration 242/356 Training loss: 2.3721 2.4906 sec/batch
Epoch 2/2  Iteration 243/356 Training loss: 2.3707 2.2887 sec/batch
Epoch 2/2  Iteration 244/356 Training loss: 2.3697 2.4664 sec/batch
Epoch 2/2  Iteration 245/356 Training loss: 2.3684 2.3931 sec/batch
Epoch 2/2  Iteration 246/356 Training loss: 2.3666 2.4584 sec/batch
Epoch 2/2  Iteration 247/356 Training loss: 2.3650 2.4322 sec/batch
Epoch 2/2  Iteration 248/356 Training loss: 2.3637 2.3503 sec/batch
Epoch 2/2  Iteration 249/356 Training loss: 2.3626 2.4665 sec/batch
Epoch 2/2  Iteration 250/356 Training loss: 2.3614 2.3181 sec/batch
Epoch 2/2  Iteration 251/356 Training loss: 2.3601 2.4767 sec/batch
Epoch 2/2  Iteration 252/356 Training loss: 2.3586 2.3571 sec/batch
Epoch 2/2  Iteration 253/356 Training loss: 2.3572 2.3802 sec/batch
Epoch 2/2  Iteration 254/356 Training loss: 2.3564 2.4256 sec/batch
Epoch 2/2  Iteration 255/356 Training loss: 2.3549 2.3070 sec/batch
Epoch 2/2  Iteration 256/356 Training loss: 2.35

In [17]:
# Print all saved checkpoints
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i356_l512_v2.120.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512_v2.399.ckpt"
all_model_checkpoint_paths: "checkpoints\\i356_l512_v2.120.ckpt"

In [18]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i356_l512_v2.120.ckpt'

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [19]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [20]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [21]:
# Change the name of latest checkpoint accordingly
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Ok ") # starting word = 'Ok '
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i356_l512_v2.120.ckpt
Ok heel at her thee withedere wast thas
sur toled wat it tho she paned to tho ghe staid th ille te tout to the
treast one him and wo te perined shis the wand sald wat he taill to the tat ours oferton tha dortely on tore to the wares, and in tha tintir, she tal wind too the stitel withe trie h ofrad the wat had why than ig hit le till to tele, and who dad to head war southell wall to the tall to to the wind an tee war that whith the past is tely the wall tor the comed ane tor whal wot his was ald, and the wathel tit or worly ter tout the soling an tout ther sionted tha lithing tha ding to har thy toll the red on he persenen to to lear thas shomring on the pand that ald il hor thing to sat to s out and atrensing, and tore the sad the site tho gering astererind,"
"Yould and ate to hes in whot will wo sering the tast to win t ant a tele at to lo the simen toorsintereraty thairs ittored to cherstere hat herres the s